In [23]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate

import mujoco_funcs as mj_funcs
import mjcf_models as mj_models
import visualize_mj_funcs as mj_vis

import dyn_functions as dyn
import visualize_dyn_funcs as vis_dyn
import gen_ctrl_funcs as gen_ctrl
import analyze_ilqr_output_funcs as analyze


In [24]:
np.set_printoptions(precision=3, suppress=True, linewidth=100)

mjcf_model  = mj_models.create_MJCF_double_pend_m_d_mod(1,1,1)
model = mujoco.MjModel.from_xml_string(mjcf_model)
data = mujoco.MjData(model)

In [34]:
dt = 0.05
len_seq = 200
time_vec = np.arange(0,len_seq*dt, dt)
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)
x_init = np.array([0.0,0.0,0.0, 0.0])
u_vec = np.array([10.0,0.0]) 
x_seq = np.zeros((len_seq, 4))
x_seq[0]= x_init

In [35]:

nu = model.nu
nx = model.nv    
eps = 1e-6
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
flg_centered = False
model.opt.timestep = dt
data.ctrl = u_vec
mujoco.mj_forward(model,data)

#initial linearization and predictiuon
state_vec = np.hstack((data.qpos.copy(), data.qvel.copy())).reshape(-1,1) 
control_vec = np.array(data.ctrl).reshape(-1,1)
A, B = mj_funcs.linearize_mujoco_state_and_control(model, data, eps, flg_centered)    
# mujoco.mjd_transitionFD(model, data, eps, flg_centered, A, B, None, None)
state_next_pred = state_next_pred = (A @ state_vec) + (B @ control_vec)


In [37]:
dpend_sys = dyn.double_pend_dyn(g=9.81, m1=m_bar, moi1=moi, d1=h_bar/2, l1=h_bar, 
                                        m2=m_bar, moi2=moi, d2=h_bar/2, l2=h_bar, 
                                        b1=0.0, b2=0.0, shoulder_act=True, elbow_act=True)
disc_dyn_func = lambda x, u: gen_ctrl.step_rk4(dpend_sys.cont_dyn_func, dt, x, u)


In [38]:
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(model, data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_init, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.969  0.031  0.05   0.   ]
 [ 0.042  0.885  0.     0.05 ]
 [-0.63   0.627  1.     0.   ]
 [ 0.838 -2.301  0.     1.   ]]
A dyn: 
 [[ 0.968  0.016  0.049  0.   ]
 [ 0.048  0.957  0.001  0.049]
 [-1.299  0.679  0.965  0.019]
 [ 1.967 -1.777  0.055  0.951]]
B mujoco: 
 [[ 0.004 -0.011]
 [-0.011  0.034]
 [ 0.085 -0.213]
 [-0.213  0.682]]
B dyn: 
 [[ 0.002 -0.005]
 [-0.003  0.012]
 [ 0.083 -0.207]
 [-0.124  0.456]]


In [ ]:
pot_energy_seq = np.zeros((len_seq,1))
kin_energy_seq = np.zeros((len_seq,1))
tot_energy_seq = np.zeros((len_seq,1))
pot_energy_seq[0] = dpend_sys.calculate_potential_energy(x_init)
kin_energy_seq[0] = dpend_sys.calculate_kinetic_energy(x_init)
tot_energy_seq[0] = dpend_sys.calculate_total_energy(x_init)